## Intro and summary of script

When PMF finalists access the Talent Management System (TMS), they eventually reach a webpage with a table of every currently open position. Each row doubles as a javascript button, which when clicked brings the finalist to a page with the full description of that particular job.

Using a web browsers 'inspect element' function, you can quickly see a pattern in those buttons. Each is uniquely named after the opportunity number (eg 217 for posting PMF-2017-0217). Because of that pattern, a webscraper can easily be written to 'click' each button in turn. Once a button is clicked, the HTML from the page with the detailed description may then be scraped for local storage.

This notebook has 2 basic parts. The first part uses the webscraper Selenium to gather PMF position information for local storage. The script is not purely automated. When Selenium initially opens the url, the user must log into the TMS, navigate to the page with the table listing every opportunity, and set the page to display all available jobs. Additionally, a table of the available jobs to be saved as an excel file before running the scraping portion of the script.

During scraping the HTMl content of each individual job posting is saved in a Pandas dataframe. Afterwards, some clunky code pulls out useful bits of information (job title, location, closing date, etc) into different columns in the dataframe before saving the dataframe as a CSV file locally.

The 2nd part of this notebook opens that locally saved CSV file and does some additional processing. Most importantly, it breaks up each posting by locations. For example, if an opportunity has 3 available positions in DC and 1 available position in CO, this section of the script breaks those into 2 different rows. A bit of additional code does a boolean test for the possible starting grade level.

As this is a Python notebook, I will be standing on the shoulders of giants by using clever packages and modules others have built. I import them here.

In [1]:
from selenium import webdriver #selenium is used to interact with the webpage, so the program can 'click' buttons.
#also make sure to have the application geckodriver in system path. Selenium needs this to function.
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary #This will let the program open the webpage on a new Firefox window.
import sys #This is only used to assign a location to my path. The location of the application geckodriver.
from bs4 import BeautifulSoup #BeautifulSoup is used to parse the HTML of the downloaded website to find the particular information desired.
import time #I will need to delay the program to give the webpage time to open. time will be used for that.
import pandas as pd #the data will be saved locally as a csv file. Pandas is a nice way to write/read/work with those files.
import numpy as np #to do math
import matplotlib.pyplot as plt #To make plots
import seaborn as sns #To make plots in a different way
from datetime import timedelta #for accurate plotting of dates%matplotlib inline
from datetime import datetime
%matplotlib inline

## Part 1: Scraping the data

This script uses the selenium package to interact with javascript buttons in an opened firefox browser window. Something that is not always obvious in the selenium docs is that a secondary application, geckodriver, needs to be in the system path to have selenium work properly. This first little bit of code just makes sure that the location of geckodriver is in the path.

In [2]:
sys.path
#sys.path.append('C:\\Users...\\folder_with_geckodriver')
sys.path.append('C:\\Users\\Kyle\\Documents\\Blog Posts\\PMF\\PMF positions')
#If this fails, try putting geckodriver in same working folder as notebook.

Now I can have Selenium open a Firefox browser using the url of the PMF TMS listings.

In [3]:
url='https://apply.pmf.gov/opportunity_search_results.aspx'
binary = FirefoxBinary('C:\\Program Files (x86)\\Mozilla Firefox\\firefox.exe')
driver = webdriver.Firefox(firefox_binary=binary)
driver.get(url)

If everying is behaving correctly, a new Firefox browser window should have opened at the log in page for the TMS. Before continuing script, go to that browser window, log in, navigate to the table of listings, and set up the table to display all open postings.

This next block scrapes the opportunity numbers from the TMS and records the date of the scraping.

In [4]:
html = driver.page_source #downloading html of individual job description
soup = BeautifulSoup(html,'lxml') #decoding the html into a workable form.
a=soup.find(id='container_date').text.replace('\xa0','')
b=datetime.strptime(a,'%A, %B %d, %Y')
date=b.strftime('%y%m%d')
keys=[];
a=soup.find('main')
for elm in a.find_all('a'):
    if 'PMF-2017-' in elm.text:
        keys.append(int(elm.text[-4:]))

Finally, it's time to let this script interact with the TMS. 

The first step is to initialize a pandas dataframe with the information from the keys. This dataframe will be where the HTMl of each posting's individual webpage is also saved.

The for loop scrapes the data, jumping back and forth between the listings and individual job postings, and with some delay built in to let the browser finish opening a new page before the script continues. You can watch the Firefox browser window while this is happening, which can be very satisfying.

In [6]:
df=pd.DataFrame()
df['listing']=keys
df['info']='1'
for elm in keys:
    #below: each javascript button associated with every row of the TMS is named after the job number
    #This for loop 'clicks' on each button in turn by making use of that pattern.
    driver.find_element_by_xpath('//a[@href="javascript:goSubmit(%s);"]'%(int(elm))).click()
    time.sleep(2) #delay to let the page load
    html = driver.page_source #downloading html of individual job description
    soup = BeautifulSoup(html,'lxml') #decoding the html into a workable form.
    data=soup.find('main') #side stepping all the CSS information from the webpage
    test=[]
    #below: this little loop finds the job description part of the main HTML section and
    #saves it as a string in df
    for row in data:
        if 'Announcement Number' in str(row):
            test.append(row)
    df.loc[df.loc[:,'listing']==elm,'info']=str(test[0])
    driver.find_element_by_xpath('//button[text()="Back"]').click() #return to TMS table and prepare to repeat.
    time.sleep(2) #delay to let the page load

Now all the data is locally captured, although the HTML formal is not very easy to read.

This online service will convert that html (first copy/paste into a notepad and save as .html) into pdf with only minor transcription errors: http://pdfcrowd.com/#convert_by_upload

A bit of clunky code pulls out some of the standard information into new columns for easier analysis.

In [7]:
#test=pd.DataFrame();
#test['label']=range(0,10)
for x in range(0,len(df)):
    a=BeautifulSoup(df.loc[x,'info'], "html.parser")
    b=a.text.split('\n')
    df.loc[x,'Position Title']=b[b.index('Position Title')+1]
    df.loc[x,'Agency/Sub-Agency']=b[b.index('Agency/Sub-Agency')+1]
    df.loc[x,'Opening Date']=b[b.index('Opening Date')+1]
    df.loc[x,'Closing Date']=b[b.index('Closing Date')+1]
    df.loc[x,'Positions']=b[b.index('Number of Positions and Location(s)')+1:b.index('Job Series and Occupational Group')]
    df.loc[x,'Series']=b[b.index('Job Series and Occupational Group')+1]
    df.loc[x,'Grade Level']=b[b.index('Pay Plan, Grade Level, and Salary Range')+1:b.index('Promotion Potential')]
    df.loc[x,'Promotion Potential']=b[b.index('Promotion Potential')+1]

In [8]:
df.tail(3)

,listing,info,Position Title,Agency/Sub-Agency,Opening Date,Closing Date,Positions,Series,Grade Level,Promotion Potential
16,479,"<div class=""row"">\n<div class=""col-sm-10"">\n<d...",Grant Program Specialist,"Department of Justice / Offices, Boards and Di...",5/5/2017,5/19/2017,"[5 in Washington DC, DC]",0301 - Miscellaneous Administration And Program,"[GS-09$549,72 - $71,467]",GS-13
17,480,"<div class=""row"">\n<div class=""col-sm-10"">\n<d...",Trade Compliance Analyst,"Department of Commerce / Commerce, Internation...",5/8/2017,5/24/2017,"[7 in Washington DC, DC]",1140 - Trade Specialist,"[GS-09, GS-11, GS-12$54,972 - $86,460]",GS-13
18,481,"<div class=""row"">\n<div class=""col-sm-10"">\n<d...",Conciliation Specialist,"Department of Justice / Offices, Boards and Di...",5/5/2017,5/19/2017,"[1 in Miami, FL1 in Philadelphia, PA]",0301 - Miscellaneous Administration And Program,"[GS-09, GS-11, GS-12$51,837 - $98,669]",GS-13


Finally, this dataframe is locally saved.

In [9]:
date

'170508'

In [10]:
df.to_csv('listingsSnapShot%s.csv'%(date),index=False)

In [11]:
len(df)

19